# OpenAI Functions 

Allow you to format the output of an llm to input to a function. Meaning allows you to call functions from outputs of LLMs.

Below is a traditional [openai function for function calling](https://platform.openai.com/docs/guides/function-calling).

In [1]:
import os

In [2]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

We can bind this function to a model using the `.bind` method from the ChatOpenAI object (or any `ChatModel` object in langchain for that matter.)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


prompt = ChatPromptTemplate.from_messages(
    [
     ("human", "{input}")   
    ]
)

model = ChatOpenAI(temperature=0).bind(functions=functions)

now we can create our runnable with:

In [4]:
# is the same as saying chain = prompt | model
runnable = prompt | model

and we can call it:

In [5]:
runnable.invoke({"input": "What's the weather like in Lisbon?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "LIS"\n}', 'name': 'weather_search'}})

Let's update the `functions` variable and add another function.

Let's say search for relevant resources given a query.

In [6]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
    {
      "name": "resources_search",
      "description": "Search for relevant resources given a query",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "The query describing the subject of the resources"
          },
        },
        "required": ["query"]
      }
    }
  ]

Now we can take that same model and bind these functions to it:

In [7]:
model = model.bind(functions=functions)

In [8]:
runnable = prompt | model

In [9]:
runnable.invoke({"input": "What are the most useful resources of the langchain Python library?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "langchain Python library"\n}', 'name': 'resources_search'}})

What if we wanted the model to put a json object rather than some string or some object?

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
import json

llm_chat = ChatOpenAI(temperature=0)
output_parser = StrOutputParser()

simple_chain = llm_chat | output_parser | json.loads

simple_chain.invoke("What are fun five names for dogs?The output should be a json object.")

{'names': ['Buddy', 'Luna', 'Max', 'Bailey', 'Coco']}

Below we add a list of other runnables. 

If there is an error it will go through the list of fallback chains (meaning, other ways of processing the input in a useful way.)

In [3]:
final_chain = simple_chain.with_fallbacks([simple_chain])

Now, let's add a prompt template to this, and explore the `.batch` option of a runnable.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

template = "Create a list of 3 exercises to drill this concept: {concept}"

prompt = ChatPromptTemplate.from_template(template)

llm_chat = ChatOpenAI(temperature=0)

output_parser = StrOutputParser()

chain = prompt | llm_chat | output_parser

chain.batch([{"concept": "derivatives"}, {"concept": "linear regression"}])

['1. Find the derivative of the function f(x) = 3x^2 + 5x - 2 using the power rule.\n2. Find the derivative of the function g(x) = sin(x) + cos(x) using the sum rule.\n3. Find the derivative of the function h(x) = e^x * ln(x) using the product rule.',
 '1. Simple linear regression practice: Start by generating a dataset with two variables, X and Y, that have a linear relationship. Use a tool like Python or R to perform simple linear regression on this dataset and interpret the results.\n\n2. Multiple linear regression practice: Create a dataset with multiple independent variables and one dependent variable that have a linear relationship. Use a statistical software package to perform multiple linear regression on this dataset and analyze the coefficients of the independent variables.\n\n3. Residual analysis exercise: Take a dataset that has been fitted with a linear regression model and analyze the residuals. Plot the residuals against the predicted values to check for patterns or hete